<a href="https://colab.research.google.com/github/Rryyuuu/code-project/blob/main/%EC%A0%95%EC%82%AC%EC%98%81%EC%83%81%EC%A0%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


필요한 패키지 설치

In [2]:
!apt-get update
!apt-get install -y gdal-bin python3-gdal
!pip install rasterio

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,519 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,860 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Package

사용할 모듈 임포트

In [2]:
from osgeo import gdal
from osgeo import osr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from pyproj import Transformer
import math

함수

In [4]:
def RFM_Open_IKONOS_RPC(rpc_filename):
    """
    Read IKONOS RPC file and return a dictionary with RPC parameters.

    Parameters
    ----------
    rpc_filename : str
        IKONOS RPC text filename

    Returns
    -------
    RPC : dict
        Dictionary containing:
            'line_off', 'samp_off', 'X_off', 'Y_off', 'Z_off'
            'line_scale', 'samp_scale', 'X_scale', 'Y_scale', 'Z_scale'
            'a', 'b', 'c', 'd'  (each NumPy array of length 20)
    """

    # Initialize RPC dict
    RPC = {
        'line_off': None,
        'samp_off': None,
        'X_off':    None,
        'Y_off':    None,
        'Z_off':    None,
        'line_scale': None,
        'samp_scale': None,
        'X_scale':    None,
        'Y_scale':    None,
        'Z_scale':    None,
        # 20개 계수 each
        'a': np.zeros(20, dtype=float),
        'b': np.zeros(20, dtype=float),
        'c': np.zeros(20, dtype=float),
        'd': np.zeros(20, dtype=float),
    }

    # Read lines from the file
    with open(rpc_filename, 'r') as f:
        lines = f.readlines()

    i = 0
    total_lines = len(lines)
    while i < total_lines:
        line = lines[i].strip()

        # ---------------------
        # 1) Offset
        # ---------------------
        if 'LINE_OFF' in line:
            # 예: "LINE_OFF: +11857.62 pixels"
            # 콜론(:) 기준 분할 후, 두 번째 부분에서 숫자만 파싱
            # split(':') => ["LINE_OFF", " +11857.62 pixels"]
            # 다시 split() => ["+11857.62", "pixels"]
            val_str = line.split(':')[1].split()[0]  # '+11857.62'
            RPC['line_off'] = float(val_str)

        elif 'SAMP_OFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['samp_off'] = float(val_str)

        elif 'LAT_OFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['Y_off'] = float(val_str)

        elif 'LONG_OFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['X_off'] = float(val_str)

        elif 'HEIGHT_OFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['Z_off'] = float(val_str)

        # ---------------------
        # 2) Scale
        # ---------------------
        elif 'LINE_SCALE' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['line_scale'] = float(val_str)

        elif 'SAMP_SCALE' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['samp_scale'] = float(val_str)

        elif 'LAT_SCALE' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['Y_scale'] = float(val_str)

        elif 'LONG_SCALE' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['X_scale'] = float(val_str)

        elif 'HEIGHT_SCALE' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['Z_scale'] = float(val_str)

        # ---------------------
        # 3) LINE_NUM_COEFF => a
        # ---------------------
        elif 'LINE_NUM_COEFF' in line:
            # 첫 줄(예: "LINE_NUM_COEFF_1: +2.50572e-04")
            val_str = line.split(':')[1].split()[0]
            RPC['a'][0] = float(val_str)
            # 다음 19줄을 더 읽어 각각 a[1..19]에 채움
            for j in range(1, 20):
                i += 1
                next_line = lines[i].strip()
                val_str = next_line.split(':')[1].split()[0]
                RPC['a'][j] = float(val_str)

        # ---------------------
        # 4) LINE_DEN_COEFF => b
        # ---------------------
        elif 'LINE_DEN_COEFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['b'][0] = float(val_str)
            for j in range(1, 20):
                i += 1
                next_line = lines[i].strip()
                val_str = next_line.split(':')[1].split()[0]
                RPC['b'][j] = float(val_str)

        # ---------------------
        # 5) SAMP_NUM_COEFF => c
        # ---------------------
        elif 'SAMP_NUM_COEFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['c'][0] = float(val_str)
            for j in range(1, 20):
                i += 1
                next_line = lines[i].strip()
                val_str = next_line.split(':')[1].split()[0]
                RPC['c'][j] = float(val_str)

        # ---------------------
        # 6) SAMP_DEN_COEFF => d
        # ---------------------
        elif 'SAMP_DEN_COEFF' in line:
            val_str = line.split(':')[1].split()[0]
            RPC['d'][0] = float(val_str)
            for j in range(1, 20):
                i += 1
                next_line = lines[i].strip()
                val_str = next_line.split(':')[1].split()[0]
                RPC['d'][j] = float(val_str)

        i += 1  # 다음 줄로 이동

    return RPC



In [6]:
def load_satellite_image(image_path):
    """
    단일 밴드 위성 이미지를 GDAL을 사용해 불러옵니다.

    매개변수:
        image_path (str): 위성 이미지 파일 경로.

    반환값:
        tuple: (위성 이미지 데이터, 이미지 폭, 이미지 높이).
    """
    dataset = gdal.Open(image_path, gdal.GA_ReadOnly)
    if not dataset:
        raise FileNotFoundError(f"이미지를 로드할 수 없습니다: {image_path}")

    # 단일 밴드 이미지 데이터 불러오기
    image_data = dataset.ReadAsArray()  # GDAL은 단일 밴드에서 (행, 열) 순서 반환
    image_width = dataset.RasterXSize
    image_height = dataset.RasterYSize

    print(f"위성 이미지 로드 완료: {image_path}")
    print(f"크기: {image_width}x{image_height}")
    print(f"가로: {image_width}")
    print(f"세로: {image_height}")

    return image_data, image_width, image_height


In [ ]:
def RFM_Image2NormalImage(RPC, img):
    """
    RFM_Image2NormalImage:
      - Converts raw image coordinates (col, row) to normalized image coords (s, l).

    Parameters
    ----------
    RPC : dict
        A dictionary containing:
            'line_off', 'samp_off',
            'line_scale', 'samp_scale',
            ...
    img : array-like of shape (N, 2) or (2,)
        Image coordinate(s) [col, row].
        - If shape is (N, 2), it means N points.
        - If just one point, can be [col, row] or (col, row).

    Returns
    -------
    nimg : ndarray of shape (N, 2)
        Normalized image coordinate(s) [s, l], where
          s = (col - samp_off) / samp_scale
          l = (row - line_off) / line_scale
    """

    # Ensure `img` is a NumPy array of shape (N, 2)
    img_arr = np.array(img, dtype=float, ndmin=2)  # at least 2D

    # col = img_arr[:,0], row = img_arr[:,1]
    # normalized sample (s) and line (l)
    s = (img_arr[:, 0] - RPC['samp_off']) / RPC['samp_scale']
    l = (img_arr[:, 1] - RPC['line_off']) / RPC['line_scale']

    # Stack into (N,2) => [s, l]
    nimg = np.column_stack((s, l))
    return nimg


def RFM_Image2Object_fixed_height(RPC, img_pnt, height):
    """
    RFM_Image2Object_fixed_height:
      - Takes an RPC dict, image coordinate (col, row), and a fixed ground height.
      - Returns the horizontal ground coordinate [X, Y, height], found by iterative refinement.

    Parameters
    ----------
    RPC : dict
      A dictionary containing fields:
        'line_off', 'samp_off',
        'X_off', 'Y_off', 'Z_off',
        'line_scale', 'samp_scale',
        'X_scale', 'Y_scale', 'Z_scale',
        'a', 'b', 'c', 'd' (each is an array of length 20)

    img_pnt : (2,) or list/tuple of length 2
      [col, row] in image space

    height : float
      Ground height (e.g. in meters above sea level)

    Returns
    -------
    obj_pnt : ndarray of shape (3,)
      The ground coordinate [X, Y, height].
      Here X=longitude, Y=latitude (or your coordinate system) depending on the RPC definition.
    """

    # 1) Extract the RPC coefficients
    a = RPC['a']  # line numerator
    b = RPC['b']  # line denominator
    c = RPC['c']  # sample numerator
    d = RPC['d']  # sample denominator

    # 2) Normalize the input image coordinate -> (s, l)
    #    s = normalized sample, l = normalized line
    nimg = RFM_Image2NormalImage(RPC, img_pnt)
    # nimg shape is (1, 2) if a single point => we can flatten to [s, l]
    s, l = nimg[0, 0], nimg[0, 1]

    # 3) 초기값 설정 (X, Y)
    #    MATLAB 코드에서는 X=RPC.X_off, Y=RPC.Y_off 로 시작
    X = RPC['X_off']
    Y = RPC['Y_off']

    # 4) 반복 변수 초기화
    tol = 1e-5             # convergence threshold
    max_iter = 100
    iter_count = 0

    dXY = np.array([9999.0, 9999.0])  # large initial difference

    # 5) 메인 반복
    while (np.any(np.abs(dXY) > tol)) and (iter_count < max_iter):
        iter_count += 1

        # 정규화된 (U, V, W)
        V = (X - RPC['X_off']) / RPC['X_scale']
        U = (Y - RPC['Y_off']) / RPC['Y_scale']
        W = (height - RPC['Z_off']) / RPC['Z_scale']

        # 20개 항 (u)
        u = np.array([
            1,
            V, U, W,
            V*U, V*W, U*W,
            V**2, U**2, W**2,
            U*V*W,
            V**3, V*(U**2), V*(W**2),
            U*(V**2), U**3, (W**2)*U, (V**2)*W, (U**2)*W, W**3
        ])

        # p1 = a·u, p2 = b·u, etc.
        p1 = a.dot(u)
        p2 = b.dot(u)
        p3 = c.dot(u)
        p4 = d.dot(u)

        # 정규화된 예측 라인/샘플
        l0 = p1 / p2  # line
        s0 = p3 / p4  # sample

        # 오차 F = [l - l0, s - s0]
        F = np.array([l - l0, s - s0])

        # 편미분 (야코비안) 계산을 위한 부분 유도항
        du_dU = np.array([
            0,
            0, 1, 0,
            V, 0, W,
            0, 2*U, 0,
            V*W,
            0, 2*U*V, 0,
            V**2, 3*U**2, 2*W*U, 0, 2*U*W, 0
        ])

        du_dV = np.array([
            0,
            1, 0, 0,
            U, W, 0,
            2*V, 0, 0,
            U*W,
            3*V**2, U**2, W**2,
            2*V*U, 0, 0, 2*V*W, 0, 0
        ])

        # dp1/dU = a·(du_dU), dp2/dU = b·(du_dU), etc.
        dp1_dU = a.dot(du_dU) / RPC['Y_scale']
        dp2_dU = b.dot(du_dU) / RPC['Y_scale']
        dp3_dU = c.dot(du_dU) / RPC['Y_scale']
        dp4_dU = d.dot(du_dU) / RPC['Y_scale']

        dp1_dV = a.dot(du_dV) / RPC['X_scale']
        dp2_dV = b.dot(du_dV) / RPC['X_scale']
        dp3_dV = c.dot(du_dV) / RPC['X_scale']
        dp4_dV = d.dot(du_dV) / RPC['X_scale']

        # dl/dX, dl/dY  (X->V, Y->U)
        dl_dX = (dp1_dV * p2 - p1 * dp2_dV) / (p2**2)
        dl_dY = (dp1_dU * p2 - p1 * dp2_dU) / (p2**2)
        ds_dX = (dp3_dV * p4 - p3 * dp4_dV) / (p4**2)
        ds_dY = (dp3_dU * p4 - p3 * dp4_dU) / (p4**2)

        # 야코비안 행렬 J
        J = np.array([
            [dl_dX, dl_dY],
            [ds_dX, ds_dY]
        ])

        # dXY = J^-1 * F
        dXY = np.linalg.inv(J).dot(F)

        # 업데이트
        X += dXY[0]
        Y += dXY[1]

    # 6) 반복 종료 및 결과
    if iter_count >= max_iter and np.any(np.abs(dXY) > tol):
        raise RuntimeError("iteration exceeded 100 without convergence")

    # 최종 지상 좌표
    obj_pnt = np.array([X, Y, height], dtype=float)
    return obj_pnt




In [8]:
def load_dsm(dsm_path):
    """
    Load a DSM (Digital Surface Model) file using GDAL.

    Parameters:
        dsm_path (str): Path to the DSM file.

    Returns:
        tuple: A tuple containing:
            - dsm_array (np.ndarray): The DSM data as a NumPy array.
            - geo_transform (tuple): GeoTransform information for the DSM.
            - projection (str): Projection information for the DSM.
    """
    # Open the DSM file
    dataset = gdal.Open(dsm_path, gdal.GA_ReadOnly)
    if not dataset:
        raise FileNotFoundError(f"Failed to load DSM from path: {dsm_path}")

    # Read DSM data into a NumPy array
    dsm_array = dataset.ReadAsArray()

    # Get GeoTransform and Projection
    geo_transform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()

    print(f"DSM loaded successfully: {dsm_path}")
    print(f"Dimensions: {dsm_array.shape}")
    print(f"GeoTransform: {geo_transform}")
    print(f"Projection: {projection}")

    return dsm_array, geo_transform, projection




In [ ]:
def latlon_to_utm52(lon_deg, lat_deg):
    """
    Convert latitude/longitude (in degrees) to UTM Zone 52N easting/northing (in meters),
    assuming a WGS84/GRS80 ellipsoid and the standard UTM scale factor 0.9996.

    Parameters
    ----------
    lon_deg : float or np.ndarray
        Longitude(s) in degrees.
    lat_deg : float or np.ndarray
        Latitude(s) in degrees (same shape as lon_deg).

    Returns
    -------
    x : np.ndarray
        Easting (meters) in UTM Zone 52N.
    y : np.ndarray
        Northing (meters) in UTM Zone 52N.
    """

    # 1) Ensure inputs are arrays
    lon = np.atleast_1d(lon_deg).astype(float)
    lat = np.atleast_1d(lat_deg).astype(float)
    npt = len(lat)

    # 2) GRS80 parameters
    a = 6378137.0
    f = 1.0 / 298.257222101
    e2 = 2.0 * f - f*f
    ep2 = e2 / (1.0 - e2)

    # 3) UTM Zone 52N constants
    #    central meridian = 129°E
    #    scale factor = 0.9996
    #    false easting = 500,000
    #    false northing = 0 (north hemisphere)
    lambda0_deg = 129.0
    lambda0 = np.deg2rad(lambda0_deg)
    k0 = 0.9996
    false_easting = 500000.0
    false_northing = 0.0

    # 4) Convert input deg -> rad
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)

    # 5) Compute some expansions
    #    M = a * [ ... ] from TM series expansion w.r.t. lat0=0
    #    (Here lat0=0 for standard UTM)
    e4 = e2*e2
    e6 = e4*e2

    A0 = 1.0 - e2/4.0 - 3.0*e4/64.0  - 5.0*e6/256.0
    B0 = 3.0*e2/8.0 + 3.0*e4/32.0    + 45.0*e6/1024.0
    C0 = 15.0*e4/256.0 + 45.0*e6/1024.0
    D0 = 35.0*e6/3072.0

    def M_fn(phi):
        return a * (
            A0*phi
            - B0*np.sin(2.0*phi)
            + C0*np.sin(4.0*phi)
            - D0*np.sin(6.0*phi)
        )

    x_out = np.zeros(npt)
    y_out = np.zeros(npt)

    for i in range(npt):
        phi = lat_rad[i]
        lam = lon_rad[i]

        # N, T, C, A
        sin_phi = np.sin(phi)
        cos_phi = np.cos(phi)
        N = a / np.sqrt(1.0 - e2*sin_phi*sin_phi)
        T = np.tan(phi)**2
        C = ep2*(cos_phi**2)
        A = (lam - lambda0)*cos_phi

        # M
        M = M_fn(phi)  # since lat0=0 => M0=0

        # easting
        A2 = A*A
        A3 = A2*A
        A4 = A3*A
        A5 = A4*A
        A6 = A5*A

        term1 = A + (1.0 - T + C)*A3/6.0
        term2 = (5.0 - 18.0*T + T*T + 72.0*C - 58.0*ep2)*(A5/120.0)

        easting = k0*N*(term1 + term2) + false_easting

        # northing
        termY2 = A2/2.0 + (5.0 - T + 9.0*C + 4.0*C*C)*A4/24.0
        termY3 = (61.0 - 58.0*T + T*T + 600.0*C - 330.0*ep2)*(A6/720.0)
        northing = k0*(M + N*np.tan(phi)*(termY2 + termY3)) + false_northing

        x_out[i] = easting
        y_out[i] = northing

    return x_out, y_out


In [ ]:
def utm_to_dsm_pixel(x, y, dsm_info):
    """
    Convert UTM (x, y) in meters to pixel coordinates (row, col) for a DSM
    that follows UTM zone 52.
    (Assuming no rotation/skew, only scale & top-left origin.)

    Parameters
    ----------
    x : float or np.ndarray
        UTM Easting
    y : float or np.ndarray
        UTM Northing
    dsm_info : dict
        A structure that includes at least:
          'x_origin' : X coord of top-left pixel corner (m)
          'y_origin' : Y coord of top-left pixel corner (m)
          'x_res'    : pixel size in X direction (m/pixel)
          'y_res'    : pixel size in Y direction (m/pixel) (often negative if north-up)

    Returns
    -------
    (row, col) : tuple of ndarray
        row, col coordinates in the DSM array
    """
    x = np.atleast_1d(x).astype(float)
    y = np.atleast_1d(y).astype(float)

    col = (x - dsm_info['x_origin']) / dsm_info['x_res']
    row = (y - dsm_info['y_origin']) / dsm_info['y_res']
    return (row, col)


In [ ]:
def latlon_to_dsm_pixel(lon_deg, lat_deg, dsm_info):
    """
    Convert lat/lon in degrees to pixel (row, col) coordinates in a DSM
    that is in UTM zone 52N.

    Parameters
    ----------
    lon_deg : float or np.ndarray
        Longitude (deg)
    lat_deg : float or np.ndarray
        Latitude (deg)
    dsm_info : dict
        Info about DSM georeference, must contain:
          'x_origin', 'y_origin', 'x_res', 'y_res'
        which define the top-left corner in UTM (x, y) and pixel resolution.

    Returns
    -------
    (row, col) : tuple of ndarray
    """

    # 1) lat/lon -> UTM (x, y) in zone 52
    x_utm, y_utm = latlon_to_utm52(lon_deg, lat_deg)

    # 2) UTM -> pixel
    row, col = utm_to_dsm_pixel(x_utm, y_utm, dsm_info)
    return (row, col)


In [ ]:
from osgeo import gdal

def subset_resample_dsm_by_pixel(
    in_dsm_path,
    out_dsm_path,
    col_min, row_min,
    col_max, row_max,
    out_pixel_size=0.5,
    resample_alg='bilinear'
):
    # 1) 입력 열기
    ds_in = gdal.Open(in_dsm_path)
    if ds_in is None:
        raise RuntimeError(f"입력 DSM 열기 실패: {in_dsm_path}")

    # 2) TranslateOptions (픽셀 범위)
    x_off = col_min
    y_off = row_min
    x_size = col_max - col_min
    y_size = row_max - row_min

    translate_opts = gdal.TranslateOptions(
        format='VRT',
        srcWin=(x_off, y_off, x_size, y_size)
    )

    vrt_ds = gdal.Translate('', ds_in, options=translate_opts)
    ds_in = None

    if vrt_ds is None:
        raise RuntimeError("픽셀 범위 잘라내기(Translate) 실패.")

    # 3) gdal.Warp 호출
    #    첫 번째 인자: 출력 파일 경로 (str)
    #    두 번째 인자: 입력 dataset (vrt_ds)
    #    나머지는 키워드 인자 (format, xRes, yRes 등)
    ds_out = gdal.Warp(
        out_dsm_path,        # destNameOrDestDS(첫 번째 인자, str)
        vrt_ds,              # srcDSOrSrcDSTab(두 번째 인자, dataset)
        format='GTiff',
        outputType=gdal.GDT_Float32,
        xRes=out_pixel_size,
        yRes=out_pixel_size,
        resampleAlg=resample_alg
    )

    vrt_ds = None

    if ds_out is None:
        raise RuntimeError("Warp 리샘플링 실패.")

    ds_out.FlushCache()
    ds_out = None

    print(f"DSM (col={col_min}~{col_max}, row={row_min}~{row_max}) -> "
          f"해상도={out_pixel_size}m 저장 완료: {out_dsm_path}")






DSM (col=2939~5794, row=3535~6364) -> 해상도=0.5m 저장 완료: /content/drive/MyDrive/gis/sample/Daejeon/resampled_dsm_50cm.tif


In [8]:
def pixel_to_utm(col, row, geo_transform):
    x_origin, x_res, x_rot, y_origin, y_rot, y_res = geo_transform
    # 회전=0 가정
    X = x_origin + col*x_res
    Y = y_origin + row*y_res
    return (X, Y)

def utm2geo(x, y, tm_lon):
    """
    UTM 좌표 (x, y, meter)와 tm_lon 값을 입력받아 경도/위도(lon, lat)를 degree로 반환.

    파라미터:
    ----------
    x: float
       UTM easting (m)
    y: float
       UTM northing (m)
    tm_lon: float
       경도 조건, MATLAB 원본에서 if tm_lon >=4 & <6 => 123, else => 129

    반환값:
    ----------
    (lon, lat): 튜플 (float, float)
       경도(lon), 위도(lat) in degree
    """

    degrad = 180.0 / math.pi  # (라디안->도 변환)
    # GRS80 파라미터
    a = 6378137.0
    f = 1.0 / 298.257222101
    e2 = 2.0 * f - f * f
    e = math.sqrt(e2)
    ep2 = e2 / (1.0 - e2)

    # MATLAB 원본 로직:
    # if tm_lon >=4 && tm_lon<6 => cmd=123; else cmd=129;
    if tm_lon >= 4 and tm_lon < 6:
        cmd = 123.0
    else:
        cmd = 129.0

    cmm = 0.0
    cms = 0.0
    olatd = 0.0
    olatm = 0.0
    olats = 0.0
    scale_cm = 0.9996

    false_easting = 500000.0
    false_northing = 0.0

    # 유도된 항들
    b = a * (1.0 - f)

    # 위도/경도 원점(위도=0, 경도=cmd°) => rad로 변환
    phi0 = olatd + olatm / 60.0 + olats / 3600.0
    lambda0 = cmd + cmm / 60.0 + cms / 3600.0
    phi0 = math.radians(phi0)
    lambda0 = math.radians(lambda0)

    # false easting/northing 제거
    X = x - false_easting
    Y = y - false_northing

    # e4, e6 등 계산
    e4 = e2 * e2
    e6 = e4 * e2

    # 아래 e1은 1 - sqrt(1-e^2) / 1 + sqrt(1-e^2) 형태
    tmp1 = 1.0 - math.sqrt(1.0 - e2)
    tmp2 = 1.0 + math.sqrt(1.0 - e2)
    e1 = tmp1 / tmp2
    e1_2 = e1 * e1
    e1_3 = e1_2 * e1
    e1_4 = e1_3 * e1

    tmp1_ = 1.0 - e2 / 4.0 - 3.0 * e4 / 64.0 - 5.0 * e6 / 256.0
    tmp2_ = 3.0 * e2 / 8.0 + 3.0 * e4 / 32.0 + 45.0 * e6 / 1024.0
    tmp3_ = 15.0 * e4 / 256.0 + 45.0 * e6 / 1024.0
    tmp4_ = 35.0 * e6 / 3072.0

    M0 = a * (tmp1_ * phi0
              - tmp2_ * math.sin(2.0 * phi0)
              + tmp3_ * math.sin(4.0 * phi0)
              - tmp4_ * math.sin(6.0 * phi0))

    k0 = scale_cm
    M = M0 + Y / k0

    # mu
    tmp1_ = 1.0 - e2 / 4.0 - 3.0 * e4 / 64.0 - 5.0 * e6 / 256.0
    mu = M / (a * tmp1_)

    # foot point latitude (phi1)
    tmp1_ = 3.0 * e1 / 2.0 - 27.0 * e1_3 / 32.0
    tmp2_ = 21.0 * e1_2 / 16.0 - 55.0 * e1_4 / 32.0
    tmp3_ = 151.0 * e1_3 / 96.0
    tmp4_ = 1097.0 * e1_4 / 512.0

    phi1 = (mu
            + tmp1_ * math.sin(2.0 * mu)
            + tmp2_ * math.sin(4.0 * mu)
            + tmp3_ * math.sin(6.0 * mu)
            + tmp4_ * math.sin(8.0 * mu))

    # 중간 계산
    C1 = ep2 * (math.cos(phi1) ** 2)
    T1 = (math.tan(phi1) ** 2)
    tmp1__ = math.sqrt(1.0 - e2 * (math.sin(phi1) ** 2))
    N1 = a / tmp1__
    tmp2__ = tmp1__ ** 3
    R1 = a * (1.0 - e2) / tmp2__

    D = X / (N1 * k0)
    D2 = D * D
    D3 = D2 * D
    D4 = D3 * D
    D5 = D4 * D
    D6 = D5 * D

    # 위도(lat) = phi1 - N1*tan(phi1)/R1 * [ ... ]
    tmp1_ = N1 * math.tan(phi1) / R1
    tmp2_ = D2 / 2.0
    tmp3_ = ((5.0 + 3.0 * T1 + 10.0 * C1 - 4.0 * (C1 ** 2) - 9.0 * ep2)
             * D4 / 24.0)
    tmp4_ = (61.0 + 90.0 * T1 + 298.0 * C1
             + 45.0 * (T1 ** 2)
             - 252.0 * ep2
             - 3.0 * (C1 ** 2)) * (D6 / 720.0)

    lat_rad = phi1 - tmp1_ * (tmp2_ - tmp3_ + tmp4_)

    # 경도(lon) = lambda0 + [D - (1+2T1+C1)*D3/6 + ... ] / cos(phi1)
    tmp1_ = D - (1.0 + 2.0 * T1 + C1) * D3 / 6.0
    tmp2_ = (5.0 - 2.0 * C1 + 28.0 * T1 - 3.0 * (C1 ** 2)
             + 8.0 * ep2 + 24.0 * (T1 ** 2)) * (D5 / 120.0)

    lon_rad = lambda0 + (tmp1_ + tmp2_) / math.cos(phi1)

    # rad -> degree
    lat_deg = lat_rad * degrad
    lon_deg = lon_rad * degrad

    return (lon_deg, lat_deg)

def create_lon_lat_height_array(dsm_path, tm_lon):
    """
    GDAL로 DSM 파일을 읽어, 각 픽셀에 대해 (경도, 위도, 고도)를
    리스트 대신 미리 할당한 ndarray에 저장.

    반환: shape=(rows*cols, 3)인 배열
    """

    # 1) DSM 열기
    ds = gdal.Open(dsm_path)
    if ds is None:
        raise RuntimeError(f"DSM 파일 열기 실패: {dsm_path}")

    # 2) DSM 배열 읽기
    band = ds.GetRasterBand(1)
    dsm_data = band.ReadAsArray().astype(np.float32)  # (rows, cols)

    rows = ds.RasterYSize
    cols = ds.RasterXSize
    geo_transform = ds.GetGeoTransform()

    # 3) 결과를 담을 배열 사전 할당
    #     크기: (rows*cols, 3)
    N = rows * cols
    result_array = np.empty((N, 3), dtype=np.float64)

    idx = 0
    # 4) 모든 픽셀 반복 -> (lon, lat, z)를 직접 result_array에 저장
    for row in range(rows):
        for col in range(cols):
            X, Y = pixel_to_utm(col, row, geo_transform)
            lon, lat = utm2geo(X, Y, tm_lon=tm_lon)
            z = dsm_data[row, col]

            result_array[idx, 0] = lon
            result_array[idx, 1] = lat
            result_array[idx, 2] = z
            idx += 1

    ds = None
    return result_array



결과 shape: (807679500, 3)
예시 첫 5개:
 [[127.29368425  36.42726799  61.6135788 ]
 [127.29368983  36.42726807  61.6135788 ]
 [127.2936954   36.42726815  61.6135788 ]
 [127.29370098  36.42726823  61.6135788 ]
 [127.29370656  36.42726831  61.6135788 ]]


In [8]:
def RFM_Object2Image(RPC, obj_pnts):
    """
    RFM_Object2Image takes RPC (Rational Polynomial Coefficients) and ground coordinates,
    then returns the corresponding image coordinates.

    Parameters
    ----------
    RPC : dict
        A dictionary containing the RPC metadata.
        Expected keys:
            'line_off', 'samp_off', 'X_off', 'Y_off', 'Z_off'
            'line_scale', 'samp_scale', 'X_scale', 'Y_scale', 'Z_scale'
            'a', 'b', 'c', 'd'  (each an array-like of length 20)

    obj_pnts : numpy.ndarray, shape (n, 3)
        Ground coordinates [X, Y, Z], where n is the number of object points.
        Typically X=longitude, Y=latitude, Z=height (depending on your RPC convention).

    Returns
    -------
    img_pnts : numpy.ndarray, shape (n, 2)
        The image coordinates [col, row].
    """

    # Number of points
    num_pnts = obj_pnts.shape[0]

    # Unpack ground coordinates
    # Assuming obj_pnts[:, 0] -> X (longitude),
    #           obj_pnts[:, 1] -> Y (latitude),
    #           obj_pnts[:, 2] -> Z (height).
    X = obj_pnts[:, 0]
    Y = obj_pnts[:, 1]
    Z = obj_pnts[:, 2]

    # 1) Normalize Coordinates (U, V, W)
    #    U, V, W are the normalized versions of (Y, X, Z) in the original code
    U = (Y - RPC['Y_off']) / RPC['Y_scale']  # e.g., normalized latitude
    V = (X - RPC['X_off']) / RPC['X_scale']  # e.g., normalized longitude
    W = (Z - RPC['Z_off']) / RPC['Z_scale']  # normalized height

    # 2) Build polynomial combinations (20 terms)
    #    This matches the 20-term sequence in the original MATLAB code.
    #    Each row of comb_UVW has shape (n, 20).
    comb_UVW = np.column_stack([
        np.ones(num_pnts),           # 1
        V,                           # 2
        U,                           # 3
        W,                           # 4
        V*U,                         # 5
        V*W,                         # 6
        U*W,                         # 7
        V**2,                        # 8
        U**2,                        # 9
        W**2,                        # 10
        U*V*W,                       # 11
        V**3,                        # 12
        V*(U**2),                    # 13
        V*(W**2),                    # 14
        U*(V**2),                    # 15
        U**3,                        # 16
        (W**2)*U,                    # 17
        (V**2)*W,                    # 18
        (U**2)*W,                    # 19
        W**3                         # 20
    ])

    # 3) Retrieve RPC coefficients
    #    Make sure they're NumPy arrays (shape (20,)) to allow vector/matrix multiplication.
    a = np.asarray(RPC['a'])
    b = np.asarray(RPC['b'])
    c = np.asarray(RPC['c'])
    d = np.asarray(RPC['d'])

    # 4) Calculate line/sample numerators and denominators
    #    p1, p2 -> for line
    #    p3, p4 -> for sample
    # comb_UVW^T has shape (20, n), so a.dot(...) yields an array of shape (n,).
    comb_UVW_t = comb_UVW.T  # (20, n)
    p1 = a.dot(comb_UVW_t)   # shape (n,)
    p2 = b.dot(comb_UVW_t)   # shape (n,)
    p3 = c.dot(comb_UVW_t)   # shape (n,)
    p4 = d.dot(comb_UVW_t)   # shape (n,)

    # Normalized line/sample
    line_norm   = p1 / p2
    sample_norm = p3 / p4

    # 5) Un-normalize image coordinates.
    #    Typically: line -> row index, sample -> column index
    #    But the code returns [col, row].
    line_scale  = RPC['line_scale']
    samp_scale  = RPC['samp_scale']
    line_off    = RPC['line_off']
    samp_off    = RPC['samp_off']

    # Allocate output array
    img_pnts = np.zeros((num_pnts, 2), dtype=np.float64)

    # col = sample_norm * samp_scale + samp_off
    # row = line_norm   * line_scale + line_off
    img_pnts[:, 0] = sample_norm * samp_scale + samp_off
    img_pnts[:, 1] = line_norm   * line_scale + line_off

    return img_pnts




In [9]:
def rfm_object2image_chunked_mat(RPC, obj_pnts, chunk_size=1000):
    """
    RFM_Object2Image 함수를 청크 단위로 수행하되,
    결과를 리스트에 모으는 대신 처음부터 (n,2) 배열을 만들어 저장.

    Parameters
    ----------
    RPC : dict
        RPC 딕셔너리(RFM_Object2Image에 필요한 필드 포함).
    obj_pnts : ndarray of shape (n, 3)
        지상 좌표 [X, Y, Z].
    chunk_size : int
        한 번에 처리할 점 개수 (기본=1000).

    Returns
    -------
    img_pnts_all : ndarray of shape (n, 2)
        (col, row) 결과.
    """

    n = obj_pnts.shape[0]
    # 최종 결과 배열을 미리 할당
    img_pnts_all = np.empty((n, 2), dtype=np.float64)

    start_idx = 0
    while start_idx < n:
        end_idx = min(start_idx + chunk_size, n)
        chunk_pts = obj_pnts[start_idx:end_idx, :]  # shape=(chunk_size or 나머지, 3)

        # RFM_Object2Image 호출
        chunk_img = RFM_Object2Image(RPC, chunk_pts)  # shape=(end_idx - start_idx, 2)

        # 결과를 미리 할당된 위치에 저장
        img_pnts_all[start_idx:end_idx, :] = chunk_img

        start_idx = end_idx

    return img_pnts_all





img_coords shape: (807679500, 2)
예시 5개:
 [[  924.28496428 -4568.75141823]
 [  925.26165379 -4568.55972977]
 [  926.23834345 -4568.36804135]
 [  927.21503324 -4568.17635298]
 [  928.19172318 -4567.98466465]]


In [12]:
def bilinear_interpolation(gcp_L1, gcp_S1, GCP_k_img):
    """
    Perform bilinear interpolation for a given floating-point coordinate.

    Parameters:
        gcp_L1 (float): Floating-point row coordinate.
        gcp_S1 (float): Floating-point column coordinate.
        GCP_k_img (np.ndarray): 2D array representing the image data.

    Returns:
        float: Interpolated pixel value.
    """
    # Extract integer parts (top-left pixel)
    XI = int(np.floor(gcp_L1))  # Integer row index
    YI = int(np.floor(gcp_S1))  # Integer column index

    # Compute fractional parts
    t = gcp_L1 - XI  # Fractional part in the row direction
    u = gcp_S1 - YI  # Fractional part in the column direction

    # Handle boundary conditions
    if XI < 0 or YI < 0 or XI+1 >= GCP_k_img.shape[0] or YI+1 >= GCP_k_img.shape[1]:
        return 0  # Return 0 if the coordinates are out of bounds

    # Perform bilinear interpolation
    dn = ((1 - t) * (1 - u) * GCP_k_img[XI, YI] +
          t * (1 - u) * GCP_k_img[XI + 1, YI] +
          t * u * GCP_k_img[XI + 1, YI + 1] +
          (1 - t) * u * GCP_k_img[XI, YI + 1])

    return dn



In [13]:
def update_dsm_with_interpolation(image_xy, satellite_image, dsm):
    """
    위성 이미지의 보간된 값을 사용해 DSM을 업데이트합니다.

    매개변수:
        pixel_x (np.ndarray): 위성 이미지의 x 좌표 데이터.
        pixel_y (np.ndarray): 위성 이미지의 y 좌표 데이터.
        satellite_image (np.ndarray): 위성 이미지 데이터 (2D 배열).
        dsm (np.ndarray): DSM 데이터 (pixel_x, pixel_y와 동일한 크기).

    반환값:
        np.ndarray: 보간된 값으로 업데이트된 DSM.
    """
    dsm_height, dsm_width = dsm.shape
    updated_dsm = np.zeros_like(dsm, dtype=np.float64)

    # 효율성을 위해 행 단위로 처리
    for row in range(dsm_height):
        for col in range(dsm_width):
            n = (row)*dsm_width +col
            # pixel_x와 pixel_y에서 부동소수점 좌표를 가져옴
            gcp_L1 = image_xy[n][1]  # y 좌표
            gcp_S1 = image_xy[n][0]  # x 좌표

            # 잘못된 좌표(예: 음수 또는 이미지 경계 밖인 경우)는 무시
            if gcp_L1 < 0 or gcp_S1 < 0 or gcp_L1 >= satellite_image.shape[0] - 1 or gcp_S1 >= satellite_image.shape[1] - 1:
                updated_dsm[row, col] = 0
                continue

            # 양선형 보간 수행
            interpolated_value = bilinear_interpolation(gcp_L1, gcp_S1, satellite_image)

            # DSM의 해당 위치에 보간 값을 삽입
            updated_dsm[row, col] = interpolated_value

    return updated_dsm



In [15]:
def save_orthophoto_as_tiff(output_path, image_data, geo_transform, projection):
    """
    Save an orthophoto as a GeoTIFF file.

    Parameters:
        output_path (str): Path to save the GeoTIFF file.
        image_data (numpy.ndarray): Orthophoto data as a NumPy array.
        geo_transform (tuple): Geotransform parameters (6 values).
        projection (str): WKT string for the spatial reference system.
    """
    # Image dimensions
    rows, cols, bands = image_data.shape if image_data.ndim == 3 else (*image_data.shape, 1)

    # Create GeoTIFF file
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(output_path, cols, rows, bands, gdal.GDT_Byte)

    if dataset is None:
        raise RuntimeError(f"Failed to create file: {output_path}")

    # Set geotransform and projection
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)

    # Write the data band by band
    if bands == 1:  # Single-band (grayscale)
        dataset.GetRasterBand(1).WriteArray(image_data)
    else:  # Multi-band (e.g., RGB)
        for band in range(bands):
            dataset.GetRasterBand(band + 1).WriteArray(image_data[:, :, band])

    # Save and close the dataset
    dataset.FlushCache()
    dataset = None

    print(f"Orthophoto saved as GeoTIFF: {output_path}")


main

In [19]:
# make resmaple_dsm
dsm_path = "/content/drive/MyDrive/gis/sample/Daejeon/Daejeon_UTM_52.tif"
satellite_image_path = "/content/drive/MyDrive/gis/sample/Daejeon/Daejeon_img1_man.tif"
rpc_path = "/content/drive/MyDrive/gis/sample/Daejeon/Daejeon_img1_man_rpc.txt"
resampled_dsm_path = "/content/drive/MyDrive/gis/sample/Daejeon/resampled_dsm_50cm.tif"

rpc_data = RFM_Open_IKONOS_RPC(rpc_path)

image_data, image_width, image_height = load_satellite_image(satellite_image_path)
image_conner = np.array([[0,0],[image_width-1,0],[0,image_height-1],[image_width-1,image_height-1]])

nimg_pts = RFM_Image2NormalImage(rpc_data, image_conner)
print("Original image points:\n", image_conner)
print("Normalized image points:\n", nimg_pts)
ground_pt = RFM_Image2Object_fixed_height(rpc_data, image_conner[0], 291.00)
ground_pt1 = RFM_Image2Object_fixed_height(rpc_data, image_conner[1], 291.00)
ground_pt2 = RFM_Image2Object_fixed_height(rpc_data, image_conner[2], 291.00)
ground_pt3 = RFM_Image2Object_fixed_height(rpc_data, image_conner[3], 291.00)


dsm_array, geo_transform, projection = load_dsm(dsm_path)

dsm_info = {
    'x_origin':  332346.1905,  # DSM 왼쪽 위 픽셀의 UTM Easting (m)
    'y_origin':  4050368.992, # DSM 왼쪽 위 픽셀의 UTM Northing (m)
    'x_res':     4.99990971601072,       # 1m/pixel in X direction
    'y_res':    -5.000251712887424,       # -1m/pixel in Y direction => 북이 위
}


row_pt0, col_pt0 = latlon_to_dsm_pixel(ground_pt0[0], ground_pt0[1], dsm_info)

print(f"lat={ground_pt0[0]}, lon={ground_pt0[1]} -> DSM pixel: row={row_pt0}, col={col_pt0}")

row_pt1, col_pt1 = latlon_to_dsm_pixel(ground_pt1[0], ground_pt1[1], dsm_info)

print(f"lat={ground_pt1[0]}, lon={ground_pt1[1]} -> DSM pixel: row={row_pt1}, col={col_pt1}")

row_pt2, col_pt2 = latlon_to_dsm_pixel(ground_pt2[0], ground_pt2[1], dsm_info)

print(f"lat={ground_pt2[0]}, lon={ground_pt2[1]} -> DSM pixel: row={row_pt2}, col={col_pt2}")

row_pt3, col_pt3 = latlon_to_dsm_pixel(ground_pt3[0], ground_pt3[1], dsm_info)

print(f"lat={ground_pt3[0]}, lon={ground_pt3[1]} -> DSM pixel: row={row_pt3}, col={col_pt3}")



output_dsm_file = "/content/drive/MyDrive/gis/sample/Daejeon/resampled_dsm_50cm.tif"

col_min_ex = 2939
row_min_ex = 3535
col_max_ex = 5794
row_max_ex = 6364

# 실행
subset_resample_dsm_by_pixel(
    in_dsm_path=dsm_path,
    out_dsm_path=output_dsm_file,
    col_min=col_min_ex,
    row_min=row_min_ex,
    col_max=col_max_ex,
    row_max=row_max_ex,
    out_pixel_size=0.5,     # 50cm
    resample_alg='bilinear'
)


In [ ]:
resampled_dsm_path = "/content/drive/MyDrive/gis/sample/Daejeon/resampled_dsm_50cm.tif"

tm_lon_val = 10  # => Zone52
arr_n3 = create_lon_lat_height_array(resampled_dsm_path, tm_lon_val)

print("결과 shape:", arr_n3.shape)
print("예시 첫 5개:\n", arr_n3[:5])

np.save("/content/drive/MyDrive/gis/sample/Daejeon/arr_n3.npy",arr_n3)

In [ ]:
arr_n3 = np.load("/content/drive/MyDrive/gis/sample/Daejeon/arr_n3.npy")
rpc_path = "/content/drive/MyDrive/gis/sample/Daejeon/Daejeon_img1_man_rpc.txt"
rpc_data = RFM_Open_IKONOS_RPC(rpc_path)

# 청크 단위로, 하지만 결과는 한 번에 (n,2) 할당
chunk_size = 1000
img_coords = rfm_object2image_chunked_mat(rpc_data, arr_n3, chunk_size)

print("img_coords shape:", img_coords.shape)  # (10000, 2)
print("예시 5개:\n", img_coords[:5])


np.save("/content/drive/MyDrive/gis/sample/Daejeon/img_coords.npy",img_coords)

In [ ]:
img_coords = np.load("/content/drive/MyDrive/gis/sample/Daejeon/img_coords.npy")
resampled_dsm_path = "/content/drive/MyDrive/gis/sample/Daejeon/resampled_dsm_50cm.tif"
dsm_array, geo_transform, projection = load_dsm(resampled_dsm_path)

# 보간 결과로 DSM 업데이트
updated_dsm = update_dsm_with_interpolation(img_coords, image_data, dsm_array)
print("DSM 업데이트 완료!")

# Save the orthophoto
save_orthophoto_as_tiff("/content/drive/MyDrive/gis/sample/Daejeon/orthophoto.tif", updated_dsm, geo_transform, projection)